In [2]:
from collections import Counter
from itertools import product
import nli
import numpy as np
import os
import pandas as pd
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import LogisticRegression
import torch
import torch.nn as nn
import torch.utils.data
from torch_model_base import TorchModelBase
from torch_rnn_classifier import TorchRNNClassifier, TorchRNNModel
from torch_shallow_neural_classifier import TorchShallowNeuralClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
import utils
import warnings
import json
import nli_ext
from gensim import models
import gensim.downloader as gd

In [3]:
utils.fix_random_seeds()

In [4]:
GLOVE_HOME = os.path.join('data', 'glove.6B')

DATA_HOME = os.path.join("data", "nlidata")

SNLI_HOME = os.path.join(DATA_HOME, "snli_1.0")

MULTINLI_HOME = os.path.join(DATA_HOME, "multinli_1.0")

ANNOTATIONS_HOME = os.path.join(DATA_HOME, "multinli_1.0_annotations")

wordentail_filename = os.path.join(
    DATA_HOME, 'nli_wordentail_bakeoff_data.json')

In [5]:
with open(wordentail_filename) as f:
    wordentail_data = json.load(f)

In [6]:
from nltk.corpus import wordnet as wn

In [21]:
#wn.synsets('house', 'n')[0].name().split('.n')[0]

def enhance_examples_with_synonyms():
    updated_wordentail_train_data = list()
    for pair in wordentail_data['train']:
        words, relation = pair
        updated_wordentail_train_data.append(pair)
        if relation == 1:            
            syn = wn.synsets(words[0], 'n')
            if len(syn) > 0:
                for w in syn:
                    updated_wordentail_train_data.append(
                        [[words[1], w.name().split('.n')[0]], relation]
                    )

            syn = wn.synsets(words[1], 'n')
            if len(syn) > 0:
                for w in syn:
                    updated_wordentail_train_data.append(
                        [[w.name().split('.n')[0], words[0]], relation]
                    )
    return updated_wordentail_train_data

In [22]:
updated_wordentail_train_data = enhance_examples_with_synonyms()

In [23]:
len(updated_wordentail_train_data)

52928

### Representing words

In [9]:
import spacy
nlp = spacy.load('en_core_web_sm')
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [10]:
def randvec(w, n=50, lower=-1.0, upper=1.0):
    """Returns a random vector of length `n`. `w` is ignored."""
    return utils.randvec(n=n, lower=lower, upper=upper)

In [11]:
def load_glove(path):
    # Creates a dict mapping strings (words) to GloVe vectors:
    glove = utils.glove2dict(path)
    # Remove stop words
#     result = dict()
#     for w in glove.keys():
#         if w not in spacy_stopwords:
#             result[w] = glove[w]
    return glove

def load_glove50():
    return load_glove(
        os.path.join(GLOVE_HOME, 'glove.6B.50d.txt'))

def load_glove100():
    return load_glove(
        os.path.join(GLOVE_HOME, 'glove.6B.100d.txt'))

def load_glove200():
    return load_glove(
        os.path.join(GLOVE_HOME, 'glove.6B.200d.txt'))

def load_glove300():
    return load_glove(
        os.path.join(GLOVE_HOME, 'glove.6B.300d.txt'))

In [ ]:
#     glove200 = load_glove200()

#     def glove_vec200(w):
#         return glove200.get(w, randvec(w, n=200))

In [12]:
# load tje GloVe data as a dictionannary of vectors
glove50 = load_glove50()
glove100 = load_glove100()
glove200 = load_glove200()
glove300 = load_glove300()

def glove_vec50(w):
    return glove50.get(w, randvec(w, n=50))

def glove_vec100(w):
    return glove100.get(w, randvec(w, n=100))

def glove_vec200(w):
    return glove200.get(w, randvec(w, n=200))

def glove_vec300(w):
    return glove300.get(w, randvec(w, n=300))

In [13]:
gnews_path = os.path.join('data', 'GoogleNews-vectors-negative300.bin')

In [14]:
word2vec = models.KeyedVectors.load_word2vec_format(
    gnews_path, binary=True)

In [15]:
def word2vec300(w):
    result = randvec(w, n=300)
    try:
        result = word2vec[w]
    except KeyError:    
        return result

    return result

In [16]:
def vec_combined(w):
    result = glove300.get(w)
    if result is not None:
        return result
    
    result = randvec(w, n=300)
    try:
        result = word2vec[w]
    except KeyError:    
        return result

    return result

In [168]:
path = gd.load('word2vec-google-news-300', return_path=True)

[=-------------------------------------------------] 3.3% 54.9/1662.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[==------------------------------------------------] 5.1% 85.4/1662.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[===-----------------------------------------------] 7.7% 127.3/1662.8MB downloaded

...

[===============================================---] 96.0% 1596.0/1662.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[=================================================-] 98.2% 1633.3/1662.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [170]:
gd.BASE_DIR = os.path.join('data', 'gensim-data')

In [176]:
path

'data/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz'

In [134]:
def twitter(w):
    result = randvec(w, n=200)
    try:
        result = glove_twitter[w]
    except KeyError:    
        return result

    return result

##### Reweitghting

In [1]:
# to be competed and tested after assignment 4. Interesting to see if the intution will be true.

def ttest(df):
    all_sum_df = df.sum().sum()
    p_df_ij = np.outer((df.sum(axis=1) / all_sum_df), 
                       (df.sum(axis=0) / all_sum_df))
    
    return ((df / all_sum_df) - p_df_ij) / np.sqrt(p_df_ij)

### Combining words into inputs (feature representations)

In [17]:
def vec_concatenate(u, v):
    return np.concatenate((u, v))

def hypothesis_only(u, v):
    return np.array(v)

def vec_diff(u, v):
    return np.array(u - v)

def vec_max(u, v):
    return np.maximum(u, v)

def vec_conc_max(u, v):
    return np.concatenate((vec_concatenate(u, v), vec_max(u, v)))

In [14]:
vec_conc_max([10, 12, 15], [3, 5, 2])

array([10, 12, 15,  3,  5,  2, 10, 12, 15])

### Setting a baseline

In [98]:
net = TorchShallowNeuralClassifier(early_stopping=True)

In [139]:
baseline_experiment = nli.wordentail_experiment(
    train_data=wordentail_data['train'],
    assess_data=wordentail_data['dev'],
    model=net,
    vector_func=glove_vec50,
    vector_combo_func=vec_concatenate)

Stopping after epoch 133. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 1.0430300533771515

              precision    recall  f1-score   support

           0      0.875     0.942     0.907      1732
           1      0.502     0.302     0.378       334

    accuracy                          0.839      2066
   macro avg      0.689     0.622     0.642      2066
weighted avg      0.815     0.839     0.822      2066



### Experimentation with models

##### TorchShallowNeuralClassifier(early_stopping=True)

In [17]:
# experiment with the baseline TorchShallowNeuralClassifier(early_stopping=True)
# best result with GloVe 200 data set 0.644 (0: 0.896, 1: 0.392)
experiment = nli.wordentail_experiment(
    train_data=wordentail_data['train'],
    assess_data=wordentail_data['dev'],
    model=net,
    vector_func=glove_vec200,
    vector_combo_func=vec_concatenate)

Stopping after epoch 38. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 1.9960777908563614

              precision    recall  f1-score   support

           0      0.870     0.917     0.893      1732
           1      0.402     0.290     0.337       334

    accuracy                          0.816      2066
   macro avg      0.636     0.604     0.615      2066
weighted avg      0.795     0.816     0.803      2066



##### LogisticRegression(fit_intercept=True, solver='liblinear', multi_class='ovr') and param_grid

**vec_concatenate(u, v)** - Best params: {'C': 0.8, 'penalty': 'l2'} - 0.626 (0: 0.896, 1: 0.357)

**hypothesis_only(u, v)** - Best params: {'C': 0.6, 'penalty': 'l2'} - 0.629 (0: 0.898, 1: 0.359)

**vec_diff(u, v)** - Best params: {'C': 0.6, 'penalty': 'l2'} - 0.569 (0: 0.906, 1: 0.232)

**vec_max(u, v)** - Best params: {'C': 1.0, 'penalty': 'l2'} - 0.637 (0: 0.906, 1: 0.367)

**vec_overlap** - Best params: {'C': 0.4, 'penalty': 'l1'} - 0.456 (0: 0.912, 1: 0.000)

In [179]:
def fit_softmax_with_hyperparameter_search(X, y):

    mod = LogisticRegression(
        fit_intercept=True,
        solver='liblinear',
        multi_class='ovr')

    param_grid = {
        'C': [0.4, 0.6, 0.8, 1.0],
        'penalty': ['l1','l2']}

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        bestmod = utils.fit_classifier_with_hyperparameter_search(
            X, y, mod, param_grid=param_grid, cv=3)

    return bestmod

In [182]:
%%time
word__experiment_xval = nli_ext.wordentail_experiment(
    train_data=wordentail_data['train'],
    assess_data=wordentail_data['train'],
    train_func=fit_softmax_with_hyperparameter_search,
    vector_func=vec_combined,
    vector_combo_func=vec_max,
    verbose=False)

Best params: {'C': 0.6, 'penalty': 'l2'}
Best score: 0.652
CPU times: user 24 s, sys: 493 ms, total: 24.5 s
Wall time: 24.7 s


In [183]:
word_experiment_xval_model = word__experiment_xval['model']
del word__experiment_xval

In [184]:
def fit_optmised_word_experiment_xval_model(X, y):
    word_experiment_xval_model.fit(X, y)
    return word_experiment_xval_model

In [185]:
result = nli_ext.wordentail_experiment(
    train_data=wordentail_data['train'],
    assess_data=wordentail_data['dev'],
    train_func=fit_optmised_word_experiment_xval_model,
    vector_func=vec_combined,
    vector_combo_func=vec_conc_max)

              precision    recall  f1-score   support

           0      0.894     0.906     0.900      1732
           1      0.477     0.443     0.460       334

    accuracy                          0.832      2066
   macro avg      0.686     0.675     0.680      2066
weighted avg      0.827     0.832     0.829      2066



In [26]:
import torch.nn as nn

class TorchDeepNeuralClassifier(TorchShallowNeuralClassifier):
    def __init__(self, dropout_prob=0.7, **kwargs):
        self.dropout_prob = dropout_prob
        super().__init__(**kwargs)
        
    def build_graph(self):        
        return nn.Sequential(
            nn.LayerNorm(self.input_dim),
            nn.Linear(self.input_dim, self.hidden_dim),
            nn.Dropout(p=self.dropout_prob),
            self.hidden_activation,
            nn.Linear(self.hidden_dim, self.n_classes_)
        )

In [159]:
net = TorchDeepNeuralClassifier(early_stopping=True, 
                                dropout_prob=0.03,
                               hidden_dim=200)

In [160]:
vcm_experiment = nli.wordentail_experiment(
        train_data=wordentail_data['train'],
        assess_data=wordentail_data['dev'],
        model=net,
        vector_func=glove_vec200,
        vector_combo_func=lambda u, v : 
            np.concatenate((vec_concatenate(u, v), vec_max(u, v))))

Stopping after epoch 33. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 0.3293429762125015

              precision    recall  f1-score   support

           0      0.891     0.930     0.910      1732
           1      0.529     0.407     0.460       334

    accuracy                          0.846      2066
   macro avg      0.710     0.669     0.685      2066
weighted avg      0.832     0.846     0.837      2066



In [29]:
def test_neural_classifier():

#    updated_wordentail_train = updated_wordentail_train_data
    updated_wordentail_train = wordentail_data['train']
    
    vm_experiment = nli.wordentail_experiment(
        train_data=updated_wordentail_train,
        assess_data=wordentail_data['dev'],
        model=net,
        vector_func=glove_vec200,
        vector_combo_func=vec_max,
        verbose=False)

    vd_experiment = nli.wordentail_experiment(
        train_data=updated_wordentail_train,
        assess_data=wordentail_data['dev'],
        model=net,
        vector_func=glove_vec200,
        vector_combo_func=vec_diff,
        verbose=False)        
    
    ho_experiment = nli.wordentail_experiment(
        train_data=updated_wordentail_train,
        assess_data=wordentail_data['dev'],
        model=net,
        vector_func=glove_vec200,
        vector_combo_func=hypothesis_only,
        verbose=False)        

    vc_experiment = nli.wordentail_experiment(
        train_data=updated_wordentail_train,
        assess_data=wordentail_data['dev'],
        model=net,
        vector_func=glove_vec200,
        vector_combo_func=vec_concatenate,
        verbose=False)

    vcm_experiment = nli.wordentail_experiment(
        train_data=updated_wordentail_train,
        assess_data=wordentail_data['dev'],
        model=net,
        vector_func=glove_vec200,
        vector_combo_func=lambda u, v : 
            np.concatenate((vec_concatenate(u, v), vec_max(u, v))),
        verbose=False)

    vcmh_experiment = nli.wordentail_experiment(
        train_data=updated_wordentail_train,
        assess_data=wordentail_data['dev'],
        model=net,
        vector_func=glove_vec200,
        vector_combo_func=lambda u, v : 
            np.concatenate((vec_concatenate(u, v), 
                            vec_max(u, v), 
                            hypothesis_only(u, v))),
        verbose=False)    
    
    result = dict()
    result[vec_concatenate.__name__] = vc_experiment['macro-F1']
    result[hypothesis_only.__name__] = ho_experiment['macro-F1']
    result[vec_max.__name__] = vm_experiment['macro-F1']
    result[vec_diff.__name__] = vd_experiment['macro-F1']
    result["concat + max"] = vcm_experiment['macro-F1']
    result["concat + max + hypo"] = vcmh_experiment['macro-F1']
    
    
    return result

In [30]:
test_neural_classifier()

Stopping after epoch 80. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 0.01555661961901933

{'vec_concatenate': 0.63213666477322,
 'hypothesis_only': 0.5630663397515601,
 'vec_max': 0.6168334808839924,
 'vec_diff': 0.6334149543986657,
 'concat + max': 0.6774463453933357,
 'concat + max + hypo': 0.6872631588129681}

In [371]:
class TorchRNNSentenceEncoderDataset(torch.utils.data.Dataset):
    def __init__(self, prem_seqs, hyp_seqs, prem_lengths, hyp_lengths, y=None):
        self.prem_seqs = prem_seqs
        self.hyp_seqs = hyp_seqs
        self.prem_lengths = prem_lengths
        self.hyp_lengths = hyp_lengths
        self.y = y
        assert len(self.prem_seqs) == len(self.hyp_seqs)
        assert len(self.hyp_seqs) == len(self.prem_lengths)
        assert len(self.prem_lengths) == len(self.hyp_lengths)
        if self.y is not None:
            assert len(self.hyp_lengths) == len(self.y)

    @staticmethod
    def collate_fn(batch):
        batch = list(zip(*batch))
        X_prem = torch.nn.utils.rnn.pad_sequence(batch[0], batch_first=True)
        X_hyp = torch.nn.utils.rnn.pad_sequence(batch[1], batch_first=True)
        prem_lengths = torch.tensor(batch[2])
        hyp_lengths = torch.tensor(batch[3])
        if len(batch) == 5:
            y = torch.tensor(batch[4])
            return X_prem, X_hyp, prem_lengths, hyp_lengths, y
        else:
            return X_prem, X_hyp, prem_lengths, hyp_lengths

    def __len__(self):
        return len(self.prem_seqs)

    def __getitem__(self, idx):
        if self.y is None:
            return (self.prem_seqs[idx], self.hyp_seqs[idx],
                    self.prem_lengths[idx], self.hyp_lengths[idx])
        else:
            return (self.prem_seqs[idx], self.hyp_seqs[idx],
                    self.prem_lengths[idx], self.hyp_lengths[idx],
                    self.y[idx])

In [372]:
class TorchRNNSentenceEncoderClassifierModel(nn.Module):
    def __init__(self, prem_rnn, hyp_rnn, output_dim):
        super().__init__()
        self.prem_rnn = prem_rnn
        self.hyp_rnn = hyp_rnn
        self.output_dim = output_dim
        self.bidirectional = self.prem_rnn.bidirectional
        # Doubled because we concatenate the final states of
        # the premise and hypothesis RNNs:
        self.classifier_dim = self.prem_rnn.hidden_dim * 2
        # Bidirectionality doubles it again:
        if self.bidirectional:
            self.classifier_dim *= 2
        self.classifier_layer = nn.Linear(
            self.classifier_dim, self.output_dim)

    def forward(self, X_prem, X_hyp, prem_lengths, hyp_lengths):
        # Premise:
        _, prem_state = self.prem_rnn(X_prem, prem_lengths)
        prem_state = self.get_batch_final_states(prem_state)
        # Hypothesis:
        _, hyp_state = self.hyp_rnn(X_hyp, hyp_lengths)
        hyp_state = self.get_batch_final_states(hyp_state)
        # Final combination:
        state = torch.cat((prem_state, hyp_state), dim=1)
        # Classifier layer:
        logits = self.classifier_layer(state)
        return logits

    def get_batch_final_states(self, state):
        if self.prem_rnn.rnn.__class__.__name__ == 'LSTM':
            state = state[0].squeeze(0)
        else:
            state = state.squeeze(0)
        if self.bidirectional:
            state = torch.cat((state[0], state[1]), dim=1)
        return state

In [373]:
class TorchRNNSentenceEncoderClassifier(TorchRNNClassifier):

    def build_dataset(self, X, y=None):
        X_prem, X_hyp = zip(*X)
        X_prem, prem_lengths = self._prepare_sequences(X_prem)
        X_hyp, hyp_lengths = self._prepare_sequences(X_hyp)
        if y is None:
            return TorchRNNSentenceEncoderDataset(
                X_prem, X_hyp, prem_lengths, hyp_lengths)
        else:
            self.classes_ = sorted(set(y))
            self.n_classes_ = len(self.classes_)
            class2index = dict(zip(self.classes_, range(self.n_classes_)))
            y = [class2index[label] for label in y]
            return TorchRNNSentenceEncoderDataset(
                X_prem, X_hyp, prem_lengths, hyp_lengths, y)

    def build_graph(self):
        prem_rnn = TorchRNNModel(
            vocab_size=len(self.vocab),
            embedding=self.embedding,
            use_embedding=self.use_embedding,
            embed_dim=self.embed_dim,
            rnn_cell_class=self.rnn_cell_class,
            hidden_dim=self.hidden_dim,
            bidirectional=self.bidirectional,
            freeze_embedding=self.freeze_embedding)

        hyp_rnn = TorchRNNModel(
            vocab_size=len(self.vocab),
            embedding=prem_rnn.embedding,  # Same embedding for both RNNs.
            use_embedding=self.use_embedding,
            embed_dim=self.embed_dim,
            rnn_cell_class=self.rnn_cell_class,
            hidden_dim=self.hidden_dim,
            bidirectional=self.bidirectional,
            freeze_embedding=self.freeze_embedding)

        model = TorchRNNSentenceEncoderClassifierModel(
            prem_rnn, hyp_rnn, output_dim=self.n_classes_)

        self.embed_dim = prem_rnn.embed_dim

        return model

In [374]:
def get_vocab():
    vocab = wordentail_data['vocab']
    vocab.append("$UNK")
    
    return vocab

In [375]:
net = TorchRNNSentenceEncoderClassifier(
        get_vocab(),
        max_iter=1000,
        embed_dim=100,
        bidirectional=True,
        hidden_dim=100)

In [377]:
X, y = zip(*updated_wordentail_train_data)
net.fit(X, y)

X_test, y_test = zip(*wordentail_data['dev'])
predictions = net.predict(X_test)

Stopping after epoch 128. Training loss did not improve more than tol=1e-05. Final error is 3.32230406999588.

In [63]:
print(classification_report(y_test, predictions, digits=3))

              precision    recall  f1-score   support

           0      0.840     0.788     0.813      1732
           1      0.168     0.222     0.191       334

    accuracy                          0.697      2066
   macro avg      0.504     0.505     0.502      2066
weighted avg      0.731     0.697     0.713      2066



In [64]:
def run200():
    net = TorchRNNSentenceEncoderClassifier(
            get_vocab(),
            max_iter=1000,
            embed_dim=200,
            bidirectional=True,
            hidden_dim=200)

    X, y = zip(*wordentail_data['train'])
    net.fit(X, y)

    X_test, y_test = zip(*wordentail_data['dev'])
    predictions = net.predict(X_test)
    return classification_report(y_test, predictions, digits=3)

In [65]:
report200 = run200()
print(report200)

Stopping after epoch 105. Training loss did not improve more than tol=1e-05. Final error is 0.8682949841022491.

              precision    recall  f1-score   support

           0      0.842     0.845     0.844      1732
           1      0.183     0.180     0.181       334

    accuracy                          0.738      2066
   macro avg      0.513     0.512     0.513      2066
weighted avg      0.736     0.738     0.737      2066



In [66]:
def run300():
    net = TorchRNNSentenceEncoderClassifier(
            get_vocab(),
            max_iter=1000,
            embed_dim=200,
            bidirectional=True,
            hidden_dim=200)

    X, y = zip(*wordentail_data['train'])
    net.fit(X, y)

    X_test, y_test = zip(*wordentail_data['dev'])
    predictions = net.predict(X_test)
    return classification_report(y_test, predictions, digits=3)

In [67]:
report300 = run300()
print(report300)

NameError: name 'run300' is not defined

In [18]:
from sklearn.neural_network import MLPClassifier

In [342]:
mlpc_model = MLPClassifier(
    alpha=0.7, learning_rate_init=0.01, max_iter=100)

In [378]:
def fit_mlp_with_hyperparameter_search(X, y):

    mod = MLPClassifier(
        alpha=0.7, learning_rate_init=0.01, max_iter=100)

    param_grid = {
        'alpha': [0.3, 0.5, 0.7, 0.8, 0.9, 1.0],
        'learning_rate_init': [0.0001, 0.001, 0.01, 0.1 ]}

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        bestmod = utils.fit_classifier_with_hyperparameter_search(
            X, y, mod, param_grid=param_grid, cv=3)

    return bestmod

In [379]:
%%time
mlpc_experiment = nli_ext.wordentail_experiment(
    train_data=updated_wordentail_train_data,
    assess_data=wordentail_data['dev'],
    train_func=fit_mlp_with_hyperparameter_search,
    vector_func=glove_vec200,
    vector_combo_func=lambda u, v : 
        np.concatenate((vec_concatenate(u, v), vec_max(u, v)))
)

Best params: {'alpha': 0.3, 'learning_rate_init': 0.001}
Best score: 0.922
              precision    recall  f1-score   support

           0      0.914     0.827     0.868      1732
           1      0.399     0.596     0.478       334

    accuracy                          0.789      2066
   macro avg      0.656     0.711     0.673      2066
weighted avg      0.831     0.789     0.805      2066

CPU times: user 10min 9s, sys: 5.7 s, total: 10min 15s
Wall time: 10min 17s


In [380]:
mlpc_trained_model = mlpc_experiment['model']

In [381]:
def get_mlpc_trained_model(X, y):
    mlpc_trained_model.max_iter = 100
    mlpc_trained_model.fit(X, y)
    return mlpc_trained_model

In [382]:
%%time
mlpc_experiment = nli_ext.wordentail_experiment(
    train_data=updated_wordentail_train_data,
    assess_data=wordentail_data['dev'],
    train_func=get_mlpc_trained_model,
    vector_func=glove_vec200,
    vector_combo_func=lambda u, v : 
        np.concatenate((vec_concatenate(u, v), vec_max(u, v)))
)

              precision    recall  f1-score   support

           0      0.908     0.816     0.860      1732
           1      0.375     0.572     0.453       334

    accuracy                          0.777      2066
   macro avg      0.642     0.694     0.656      2066
weighted avg      0.822     0.777     0.794      2066

CPU times: user 19.4 s, sys: 253 ms, total: 19.6 s
Wall time: 19.8 s


In [384]:
mlpc_experiment = nli.wordentail_experiment(
    train_data=wordentail_data['train'],
    assess_data=wordentail_data['dev'],
    model=mlpc_trained_model,
    vector_func=glove_vec200,
    vector_combo_func=lambda u, v : 
        np.concatenate((vec_concatenate(u, v), vec_max(u, v)))
)

              precision    recall  f1-score   support

           0      0.886     0.943     0.914      1732
           1      0.559     0.371     0.446       334

    accuracy                          0.851      2066
   macro avg      0.722     0.657     0.680      2066
weighted avg      0.833     0.851     0.838      2066



/Users/antongochev/opt/miniconda3/envs/nlu/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [24]:
mlpc_model = MLPClassifier(
    alpha=0.2, learning_rate_init=0.001, max_iter=1000)

In [26]:
mlpc_experiment = nli.wordentail_experiment(
    train_data=wordentail_data['train'],
    assess_data=wordentail_data['dev'],
    model=mlpc_model,
    vector_func=vec_combined,
    vector_combo_func= lambda u, v : 
        np.concatenate((vec_concatenate(u, v), vec_max(u, v)))
)

              precision    recall  f1-score   support

           0      0.880     0.956     0.916      1732
           1      0.584     0.323     0.416       334

    accuracy                          0.853      2066
   macro avg      0.732     0.639     0.666      2066
weighted avg      0.832     0.853     0.835      2066



In [130]:
def vec_combined(w):
    result = glove200.get(w)
    if result is not None:
        return result
    
    result = None
    try:
        result = glove_twitter[w]
    except KeyError:    
        return result

    return result

non_in_list = list()
for (w1, w2), index in wordentail_data['train']:
    word = vec_combined(w1)
    if word is None:
        non_in_list.append(w1)
        
    word = vec_combined(w2)
    if word is None:
        non_in_list.append(w2)    

In [131]:
len(non_in_list)

352

##### TorchRNNClassifier 

Chained models

In [15]:
def fit_simple_chained_rnn_hp_search(X, y):
    
    model = TorchRNNClassifier(
        get_vocab(),
        hidden_dim=200,
        embed_dim=200,
        bidirectional=True,
        early_stopping=True,
        max_iter=1
    )
    
    pg = {
        'batch_size': [32, 64],
        'eta': [0.001, 0.01]
    }
    
    bestmod = utils.fit_classifier_with_hyperparameter_search(
        X, y, model, param_grid=pg, cv=3)
    
    return bestmod

In [ ]:
# %%time
# chained_rnn_xval = nli_ext.wordentail_experiment(
#     train_data=wordentail_data['train'],
#     assess_data=wordentail_data['train'],
#     train_func=fit_simple_chained_rnn_hp_search,
#     vector_func=glove_vec200,
#     vector_combo_func=vec_max)

In [201]:
chained_rnn_xval_model = chained_rnn_xval['model']
del chained_rnn_xval

In [202]:
def fit_optmised_chained_rnn_xval_model(X, y):
    chained_rnn_xval_model.max_iter = 10
    chained_rnn_xval_model.fit(X, y)
    return chained_rnn_xval_model

In [203]:
# result = nli_ext.wordentail_experiment(
#     train_data=wordentail_data['train'],
#     assess_data=wordentail_data['dev'],
#     train_func=fit_optmised_chained_rnn_xval_model,
#     vector_func=glove_vec50,
#     vector_combo_func=vec_max)

Finished epoch 10 of 10; error is 100.30287890136242

              precision    recall  f1-score   support

           0      0.838     1.000     0.912      1732
           1      0.000     0.000     0.000       334

    accuracy                          0.838      2066
   macro avg      0.419     0.500     0.456      2066
weighted avg      0.703     0.838     0.765      2066



In [16]:
net = TorchRNNClassifier(
        get_vocab(),
        hidden_dim=100,
        embed_dim=50,
        bidirectional=True,
        early_stopping=True,
        eta=0.01
    )

In [17]:
# %%time
# experiment = nli.wordentail_experiment(
#     train_data=wordentail_data['train'],
#     assess_data=wordentail_data['dev'],
#     model=net,
#     vector_func=glove_vec50,
#     vector_combo_func=vec_max)

Stopping after epoch 12. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 3.4910351037979126

              precision    recall  f1-score   support

           0      0.838     1.000     0.912      1732
           1      0.000     0.000     0.000       334

    accuracy                          0.838      2066
   macro avg      0.419     0.500     0.456      2066
weighted avg      0.703     0.838     0.765      2066

CPU times: user 16min 18s, sys: 2min 48s, total: 19min 7s
Wall time: 6min 1s


/Users/antongochev/opt/miniconda3/envs/nlu/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
